# Data Access Instructions

Data are accessible via [an object store](https://cloud.google.com/learn/what-is-object-storage) system, which enables easy, scalable web access! 

## Imports


In [1]:
import fsspec
import urllib.request
from pathlib import Path
import xradar as xd
import pyart


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



## Find Data in the Bucket

It is stored on the jetstream cloud bucket, under the `/pythia/radar/ams2025` space.

In [2]:
# Set the URL and path for the cloud
URL = "https://js2.jetstream-cloud.org:8001/"
path = f"pythia/radar/ams2025"


fs = fsspec.filesystem("s3",
                       anon=True, 
                       client_kwargs=dict(endpoint_url=URL))

fs.glob(f"{path}/*")

['pythia/radar/ams2025/CalgaryHailStorm2024',
 'pythia/radar/ams2025/OntarioDerecho2022']

### Find Data to Stream
We can easily stream the data with xradar

In [10]:
files = fs.glob("pythia/radar/ams2025/OntarioDerecho2022/*")[:1]
files

[]

In [4]:
local_files = [
    fsspec.open_local(
        f"simplecache::{URL}{i}", s3={"anon": True}, filecache={"cache_storage": "."}
    )
    for i in files
]

In [6]:
dt = xd.io.open_odim_datatree(local_files[0])
radar = dt.pyart.to_radar()

In [9]:
radar

DataTree('root', parent=None)
│   Dimensions:              ()
│   Data variables:
│       volume_number        int64 8B 0
│       platform_type        <U5 20B 'fixed'
│       instrument_type      <U5 20B 'radar'
│       time_coverage_start  <U20 80B '2022-05-21T14:54:03Z'
│       time_coverage_end    <U20 80B '2022-05-21T14:59:48Z'
│       longitude            float64 8B -81.38
│       altitude             float64 8B 365.4
│       latitude             float64 8B 43.37
│   Attributes:
│       Conventions:      ODIM_H5/V2_2
│       instrument_name:  None
│       version:          None
│       title:            None
│       institution:      None
│       references:       None
│       source:           None
│       history:          None
│       comment:          im/exported using xradar
├── DataTree('sweep_0')
│       Dimensions:            (azimuth: 360, range: 245)
│       Coordinates:
│           elevation          (azimuth) float64 3kB ...
│           time               (azimuth) datetime64[ns] 3kB 2022-05-21T14:54:06.58332...
│         * range              (range) float32 980B 250.0 750.0 ... 1.218e+05 1.222e+05
│           longitude          float64 8B ...
│           latitude           float64 8B ...
│           altitude           float64 8B ...
│         * azimuth            (azimuth) float64 3kB 0.4945 1.502 2.507 ... 358.5 359.5
│       Data variables: (12/15)
│           DBZH               (azimuth, range) float64 706kB ...
│           TH                 (azimuth, range) float64 706kB ...
│           RHOHV              (azimuth, range) float64 706kB ...
│           UPHIDP             (azimuth, range) float64 706kB ...
│           WRADH              (azimuth, range) float64 706kB ...
│           PHIDP              (azimuth, range) float64 706kB ...
│           ...                 ...
│           VRADH              (azimuth, range) float64 706kB ...
│           sweep_mode         <U20 80B ...
│           sweep_number       int64 8B ...
│           prt_mode           <U7 28B ...
│           follow_mode        <U7 28B ...
│           sweep_fixed_angle  float64 8B ...
├── DataTree('sweep_1')
│       Dimensions:            (azimuth: 360, range: 245)
│       Coordinates:
│           elevation          (azimuth) float64 3kB ...
│           time               (azimuth) datetime64[ns] 3kB 2022-05-21T14:54:21.41666...
│         * range              (range) float32 980B 250.0 750.0 ... 1.218e+05 1.222e+05
│           longitude          float64 8B ...
│           latitude           float64 8B ...
│           altitude           float64 8B ...
│         * azimuth            (azimuth) float64 3kB 0.4915 1.497 2.502 ... 358.5 359.5
│       Data variables: (12/15)
│           DBZH               (azimuth, range) float64 706kB ...
│           TH                 (azimuth, range) float64 706kB ...
│           RHOHV              (azimuth, range) float64 706kB ...
│           UPHIDP             (azimuth, range) float64 706kB ...
│           WRADH              (azimuth, range) float64 706kB ...
│           PHIDP              (azimuth, range) float64 706kB ...
│           ...                 ...
│           VRADH              (azimuth, range) float64 706kB ...
│           sweep_mode         <U20 80B ...
│           sweep_number       int64 8B ...
│           prt_mode           <U7 28B ...
│           follow_mode        <U7 28B ...
│           sweep_fixed_angle  float64 8B ...
├── DataTree('sweep_2')
│       Dimensions:            (azimuth: 360, range: 245)
│       Coordinates:
│           elevation          (azimuth) float64 3kB ...
│           time               (azimuth) datetime64[ns] 3kB 2022-05-21T14:54:36.21665...
│         * range              (range) float32 980B 250.0 750.0 ... 1.218e+05 1.222e+05
│           longitude          float64 8B ...
│           latitude           float64 8B ...
│           altitude           float64 8B ...
│         * azimuth            (azimuth) float64 3kB 0.5055 1.511 2.502 ... 358.5 359.5
│       Data va

### Download the Data Locally
We can also use the url of the data to download the data locally if neccessary!

In [8]:
for file in files:
    urllib.request.urlretrieve(f"{URL}{file}", Path(file).name)

### ARCO Data Access

Canadian data was converted into Analysis-Ready Cloud-Optimized and can be accessed as follows

In [14]:
import xarray as xr
import icechunk

Configure S3 storage for [icechunk](https://icechunk.io/en/latest/)

In [21]:
storage = icechunk.s3_storage(
    bucket='pythia',
    prefix='radar/ams2025/CASSM.zarr',
    endpoint_url='https://js2.jetstream-cloud.org:8001',
    anonymous=True,
    region='us-east-1',
    force_path_style=True
)

Creating S3 bucket connection and immutable session

In [22]:
repo = icechunk.Repository.open(storage=storage)

In [23]:
session = repo.readonly_session("main")

Opening the Radar datatree using `xarray`

In [18]:
dtree = xr.open_datatree(
    session.store, 
    engine="zarr",
    consolidated=False,
    chunks={}
)

In [19]:
dtree

<xarray.DataTree>
Group: /
├── Group: /georeferencing_correction
│       Dimensions:    (vcp_time: 56)
│       Coordinates:
│           longitude  float64 8B ...
│         * vcp_time   (vcp_time) datetime64[ns] 448B 2024-08-05T23:54:03 ... 2024-08...
│           latitude   float64 8B ...
│           altitude   float64 8B ...
├── Group: /sweep_0
│       Dimensions:            (vcp_time: 56, azimuth: 360, range: 245)
│       Coordinates:
│         * azimuth            (azimuth) float64 3kB 0.5 1.5 2.5 ... 357.5 358.5 359.5
│           elevation          (azimuth) float64 3kB dask.array<chunksize=(360,), meta=np.ndarray>
│           latitude           float64 8B ...
│           crs_wkt            int64 8B ...
│           altitude           float64 8B ...
│         * range              (range) float32 980B 250.0 750.0 ... 1.218e+05 1.222e+05
│         * vcp_time           (vcp_time) datetime64[ns] 448B 2024-08-05T23:54:03 ......
│           z                  (azimuth, range) float64 706kB dask.array<chunksize=(180, 123), meta=np.ndarray>
│           x                  (azimuth, range) float64 706kB dask.array<chunksize=(180, 123), meta=np.ndarray>
│           time               (azimuth) datetime64[ns] 3kB dask.array<chunksize=(360,), meta=np.ndarray>
│           longitude          float64 8B ...
│           y                  (azimuth, range) float64 706kB dask.array<chunksize=(180, 123), meta=np.ndarray>
│       Data variables:
│           DBZH               (vcp_time, azimuth, range) float32 20MB dask.array<chunksize=(1, 360, 245), meta=np.ndarray>
│           PHIDP              (vcp_time, azimuth, range) float32 20MB dask.array<chunksize=(1, 360, 245), meta=np.ndarray>
│           WRADH              (vcp_time, azimuth, range) float32 20MB dask.array<chunksize=(1, 360, 245), meta=np.ndarray>
│           KDP                (vcp_time, azimuth, range) float32 20MB dask.array<chunksize=(1, 360, 245), meta=np.ndarray>
│           SQIH               (vcp_time, azimuth, range) float32 20MB dask.array<chunksize=(1, 360, 245), meta=np.ndarray>
│           TH                 (vcp_time, azimuth, range) float32 20MB dask.array<chunksize=(1, 360, 245), meta=np.ndarray>
│           VRADH              (vcp_time, azimuth, range) float32 20MB dask.array<chunksize=(1, 360, 245), meta=np.ndarray>
│           ZDR                (vcp_time, azimuth, range) float32 20MB dask.array<chunksize=(1, 360, 245), meta=np.ndarray>
│           RHOHV              (vcp_time, azimuth, range) float32 20MB dask.array<chunksize=(1, 360, 245), meta=np.ndarray>
│           sweep_number       (vcp_time) float64 448B dask.array<chunksize=(1,), meta=np.ndarray>
│           UPHIDP             (vcp_time, azimuth, range) float32 20MB dask.array<chunksize=(1, 360, 245), meta=np.ndarray>
│           sweep_fixed_angle  (vcp_time) float32 224B dask.array<chunksize=(1,), meta=np.ndarray>
├── Group: /sweep_12
│       Dimensions:            (vcp_time: 56, azimuth: 720, range: 660)
│       Coordinates:
│           altitude           float64 8B ...
│           longitude          float64 8B ...
│         * azimuth            (azimuth) float64 6kB 0.25 0.75 1.25 ... 359.2 359.8
│         * range              (range) float32 3kB 250.0 750.0 ... 3.292e+05 3.298e+05
│           crs_wkt            int64 8B ...
│           elevation          (azimuth) float64 6kB dask.array<chunksize=(720,), meta=np.ndarray>
│         * vcp_time           (vcp_time) datetime64[ns] 448B 2024-08-05T23:54:03 ......
│           latitude           float64 8B ...
│           time               (azimuth) datetime64[ns] 6kB dask.array<chunksize=(720,), meta=np.ndarray>
│           z                  (azimuth, range) float64 4MB dask.array<chunksize=(180, 330), meta=np.ndarray>
│           x                  (azimuth, range) float64 4MB dask.array<chunksize=(180, 330), meta=np.ndarray>
│           y                  (azimuth, range) float64 4MB dask.array<chunksize=(180, 330), meta=np.ndarray>
│       Data variabl